<a href="https://colab.research.google.com/github/Luc1lfer/CCMACLRL_EXERCISES_COM221ML/blob/main/Exercise_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, log_loss



In [4]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")


In [5]:
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)


In [6]:
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1


In [7]:
categorical_cols = ['Sex', 'Embarked', 'Pclass']
for col in categorical_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])


In [8]:
scaler = StandardScaler()
numerical_cols = ['Age', 'Fare', 'FamilySize']
train_df[numerical_cols] = scaler.fit_transform(train_df[numerical_cols])
test_df[numerical_cols] = scaler.transform(test_df[numerical_cols])


In [9]:
train_df = train_df.drop('Name', axis=1)
test_df = test_df.drop('Name', axis=1)



In [10]:
train_df = train_df.drop('Ticket', axis=1)
test_df = test_df.drop('Ticket', axis=1)



In [11]:
train_df = train_df.drop('Cabin', axis=1)
test_df = test_df.drop('Cabin', axis=1)



In [12]:
imputer = SimpleImputer(strategy='mean')



In [13]:
X_train = imputer.fit_transform(train_df.drop('Survived', axis=1))

X_test = imputer.transform(test_df)

X_train, X_val, y_train, y_val = train_test_split(X_train, train_df['Survived'], test_size=0.2, random_state=42)



In [14]:
model = LogisticRegression()
model.fit(X_train, y_train)



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [15]:
y_pred_val = model.predict_proba(X_val)[:, 1]
log_loss_val = log_loss(y_val, y_pred_val)
print(f"Log Loss on Validation Set: {log_loss_val}")
y_pred_val_class = model.predict(X_val)
accuracy_val = accuracy_score(y_val, y_pred_val_class)
print(f"Accuracy on Validation Set: {accuracy_val}")



Log Loss on Validation Set: 0.44980609443714226
Accuracy on Validation Set: 0.7932960893854749


In [16]:
y_pred_test = model.predict_proba(X_test)[:, 1]

submission_df = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': (y_pred_test >= 0.5).astype(int)})

submission_df.to_csv('submission.csv', index=False)